In [1]:
import numpy as np
import pandas as pd
from openpyxl import load_workbook
from openpyxl.pivot.fields import Missing

In [2]:
file_path = '../data/vendas-combustiveis-m3.xlsx'

In [3]:
workbook = load_workbook(file_path)
worksheet = workbook['Plan1']

In [5]:
pivot_name = 'Tabela dinâmica3'

In [6]:
# Extract the pivot table object from the worksheet
pivot_table = [p for p in worksheet._pivots if p.name == pivot_name][0]


In [7]:
# Extract a dict of all cache fields and their respective values
fields_map = {}
for field in pivot_table.cache.cacheFields:
    if field.sharedItems.count > 0:
        # take care of cases where f.v returns an AttributeError because the cell is empty
        # fields_map[field.name] = [f.v for f in field.sharedItems._fields]
        l = []
        for f in field.sharedItems._fields:
            try:
                l += [f.v]
            except AttributeError:
                l += [""]
        fields_map[field.name] = l


In [8]:
# Extract all rows from cache records. Each row is initially parsed as a dict
column_names = [field.name for field in pivot_table.cache.cacheFields]


In [9]:
rows = []
for record in pivot_table.cache.records.r:
    # If some field in the record in missing, we replace it by NaN
    record_values = [
        field.v if not isinstance(field, Missing) else np.nan for field in record._fields
    ]

    row_dict = {k: v for k, v in zip(column_names, record_values)}

    # Shared fields are mapped as an Index, so we replace the field index by its value
    for key in fields_map:
        row_dict[key] = fields_map[key][row_dict[key]]

    rows.append(row_dict)


In [10]:
df = pd.DataFrame.from_dict(rows)
df

,COMBUSTÍVEL,ANO,REGIÃO,ESTADO,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,TOTAL
0,ÓLEO DIESEL S-10 (m3),2013.0,REGIÃO NORTE,RONDÔNIA,81453.67,3517.6,3681.7,4700.67,5339.2,6166.4,6539.65,7283.7,8082.85,7902.55,9383.15,9767.4,9088.8
1,ÓLEO DIESEL S-10 (m3),2013.0,REGIÃO NORTE,ACRE,1483.0,11202.0,363.0,410.0,536.0,607.0,740.0,756.0,971.0,1174.0,1240.0,1439.0,1483.0
2,ÓLEO DIESEL S-10 (m3),2013.0,REGIÃO NORTE,AMAZONAS,6836.3,6784.232,61443.832,3190.585,3305.0,3391.0,3637.0,4250.0,4576.0,5756.879,6228.636,6334.0,7154.2
3,ÓLEO DIESEL S-10 (m3),2013.0,REGIÃO NORTE,RORAIMA,1475.3,1502.7,1531.8,13423.7,795.4,757.2,939.8,1040.6,966.0,992.9,1027.0,1083.8,1311.2
4,ÓLEO DIESEL S-10 (m3),2013.0,REGIÃO NORTE,PARÁ,40913.48,45383.5,44013.219,41975.03,441140.785,30137.8,28146.3,31280.5,33033.05,33519.88,34321.53,37168.16,41248.336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,ÓLEO DIESEL (OUTROS ) (m3),2020.0,REGIÃO SUL,RIO GRANDE DO SUL,0.0,0.0,10.0,10.0,0.0,10.0,,,,60.0,10.0,10.0,10.0
1076,ÓLEO DIESEL (OUTROS ) (m3),2020.0,REGIÃO CENTRO-OESTE,MATO GROSSO DO SUL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,,0.0,0.0,0.0
1077,ÓLEO DIESEL (OUTROS ) (m3),2020.0,REGIÃO CENTRO-OESTE,MATO GROSSO,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,,90.0,45.0
1078,ÓLEO DIESEL (OUTROS ) (m3),2020.0,REGIÃO CENTRO-OESTE,GOIÁS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.4,,,,41.4


In [68]:
df.columns

Index(['COMBUSTÍVEL', 'ANO', 'REGIÃO', 'ESTADO', 'Jan', 'Fev', 'Mar', 'Abr',
       'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez', 'TOTAL'],
      dtype='object')

In [69]:
df.shape

(4536, 17)

In [72]:
df.dtypes

COMBUSTÍVEL     object
ANO            float64
ESTADO          object
Jan             object
Fev             object
Mar             object
Abr             object
Mai             object
Jun             object
Jul             object
Ago             object
Set             object
Out             object
Nov             object
Dez             object
dtype: object

In [71]:
cols_to_drop = ['TOTAL','REGIÃO']
df.drop(columns=cols_to_drop, inplace=True)

In [74]:
dict_m = {"Jan": 1, "Fev": 2, "Mar": 3, "Abr": 4, "Mai": 5,"Jun": 6, "Jul": 7, "Ago": 8, "Set": 9, "Out": 10, "Nov": 11, "Dez": 12}

In [75]:
df = pd.melt(df, id_vars = ["COMBUSTÍVEL", "ANO", "ESTADO"], value_vars = dict_m.keys(), var_name = "month", value_name = "volume")


In [76]:
df

,COMBUSTÍVEL,ANO,ESTADO,month,volume
0,GASOLINA C (m3),2000.0,RONDÔNIA,Jan,136073.253
1,GASOLINA C (m3),2000.0,ACRE,Jan,3358.346
2,GASOLINA C (m3),2000.0,AMAZONAS,Jan,20766.918
3,GASOLINA C (m3),2000.0,RORAIMA,Jan,3716.032
4,GASOLINA C (m3),2000.0,PARÁ,Jan,29755.907
...,...,...,...,...,...
54427,GLP (m3),2020.0,RIO GRANDE DO SUL,Dez,64045.161232
54428,GLP (m3),2020.0,MATO GROSSO DO SUL,Dez,16281.139493
54429,GLP (m3),2020.0,MATO GROSSO,Dez,18321.987319
54430,GLP (m3),2020.0,GOIÁS,Dez,46850.585145


In [78]:
df['unit'] = 'm3'

from datetime import datetime
# timestamp = datetime.now()
# timestamp_series = pd.to_datetime(timestamp)
df['created_at'] = datetime.now()

In [82]:
df.rename(columns={'COMBUSTÍVEL': 'product','ESTADO': 'uf'},inplace=True)

In [83]:
df

,product,ANO,uf,month,volume,unit,created_at
0,GASOLINA C (m3),2000.0,RONDÔNIA,Jan,136073.253,m3,2023-10-10 13:32:01.584206
1,GASOLINA C (m3),2000.0,ACRE,Jan,3358.346,m3,2023-10-10 13:32:01.584206
2,GASOLINA C (m3),2000.0,AMAZONAS,Jan,20766.918,m3,2023-10-10 13:32:01.584206
3,GASOLINA C (m3),2000.0,RORAIMA,Jan,3716.032,m3,2023-10-10 13:32:01.584206
4,GASOLINA C (m3),2000.0,PARÁ,Jan,29755.907,m3,2023-10-10 13:32:01.584206
...,...,...,...,...,...,...,...
54427,GLP (m3),2020.0,RIO GRANDE DO SUL,Dez,64045.161232,m3,2023-10-10 13:32:01.584206
54428,GLP (m3),2020.0,MATO GROSSO DO SUL,Dez,16281.139493,m3,2023-10-10 13:32:01.584206
54429,GLP (m3),2020.0,MATO GROSSO,Dez,18321.987319,m3,2023-10-10 13:32:01.584206
54430,GLP (m3),2020.0,GOIÁS,Dez,46850.585145,m3,2023-10-10 13:32:01.584206


In [84]:
df.dtypes

product               object
ANO                  float64
uf                    object
month                 object
volume                object
unit                  object
created_at    datetime64[ns]
dtype: object

In [92]:
df['year_month'] = (df['ANO'].astype(int).astype(str) + '-' + df['month'].replace(dict_m).astype(str))

In [94]:
df['year_month'] = pd.to_datetime(df['year_month'], format='%Y-%m')


In [96]:
columns_order = ['year_month','uf','product','unit','volume','created_at']
df = df[columns_order]


In [99]:
df.dtypes

year_month    datetime64[ns]
uf                    object
product               object
unit                  object
volume                object
created_at    datetime64[ns]
dtype: object

In [100]:
df

,year_month,uf,product,unit,volume,created_at
0,2000-01-01,RONDÔNIA,GASOLINA C (m3),m3,136073.253,2023-10-10 13:32:01.584206
1,2000-01-01,ACRE,GASOLINA C (m3),m3,3358.346,2023-10-10 13:32:01.584206
2,2000-01-01,AMAZONAS,GASOLINA C (m3),m3,20766.918,2023-10-10 13:32:01.584206
3,2000-01-01,RORAIMA,GASOLINA C (m3),m3,3716.032,2023-10-10 13:32:01.584206
4,2000-01-01,PARÁ,GASOLINA C (m3),m3,29755.907,2023-10-10 13:32:01.584206
...,...,...,...,...,...,...
54427,2020-12-01,RIO GRANDE DO SUL,GLP (m3),m3,64045.161232,2023-10-10 13:32:01.584206
54428,2020-12-01,MATO GROSSO DO SUL,GLP (m3),m3,16281.139493,2023-10-10 13:32:01.584206
54429,2020-12-01,MATO GROSSO,GLP (m3),m3,18321.987319,2023-10-10 13:32:01.584206
54430,2020-12-01,GOIÁS,GLP (m3),m3,46850.585145,2023-10-10 13:32:01.584206


In [104]:
len(df['product'].unique().tolist())

8

In [103]:
12*21

252

In [57]:
df['ESTADO'] = df['ESTADO'].astype('string')

In [58]:
df.dtypes

COMBUSTÍVEL     string
ANO            float64
ESTADO          string
Jan             object
Fev             object
Mar             object
Abr             object
Mai             object
Jun             object
Jul             object
Ago             object
Set             object
Out             object
Nov             object
Dez             object
dtype: object

In [59]:
df['unit'] = 'm3'


In [60]:
df['unit'] = df['unit'].astype('string')

In [61]:
from datetime import datetime
timestamp = datetime.now()
timestamp_series = pd.to_datetime(timestamp)
df['created_at'] = timestamp_series

In [62]:
df.dtypes

COMBUSTÍVEL            string
ANO                   float64
ESTADO                 string
Jan                    object
Fev                    object
Mar                    object
Abr                    object
Mai                    object
Jun                    object
Jul                    object
Ago                    object
Set                    object
Out                    object
Nov                    object
Dez                    object
unit                   string
created_at     datetime64[ns]
dtype: object

In [63]:
MONTH_DICT = {
    "Jan": 1, 
    "Fev": 2, 
    "Mar": 3, 
    "Abr": 4, 
    "Mai": 5,
    "Jun": 6, 
    "Jul": 7, 
    "Ago": 8, 
    "Set": 9, 
    "Out": 10, 
    "Nov": 11, 
    "Dez": 12
}

In [64]:
df = pd.melt(df, id_vars = ["ANO", "ESTADO", "COMBUSTÍVEL", "unit", "created_at"], value_vars = MONTH_DICT.keys(), var_name = "month", value_name = "volume")


In [49]:
df2 = pd.melt(df, id_vars = ["ANO", "ESTADO", "COMBUSTÍVEL", "unit","created_at"], var_name = "month", value_name = "volume")


In [35]:
df2

,ANO,ESTADO,COMBUSTÍVEL,unit,month,volume
0,2000.0,RONDÔNIA,GASOLINA C (m3),m3,month,Jan
1,2000.0,ACRE,GASOLINA C (m3),m3,month,Jan
2,2000.0,AMAZONAS,GASOLINA C (m3),m3,month,Jan
3,2000.0,RORAIMA,GASOLINA C (m3),m3,month,Jan
4,2000.0,PARÁ,GASOLINA C (m3),m3,month,Jan
...,...,...,...,...,...,...
108859,2020.0,RIO GRANDE DO SUL,GLP (m3),m3,volume,64045.161232
108860,2020.0,MATO GROSSO DO SUL,GLP (m3),m3,volume,16281.139493
108861,2020.0,MATO GROSSO,GLP (m3),m3,volume,18321.987319
108862,2020.0,GOIÁS,GLP (m3),m3,volume,46850.585145


In [65]:
df

,ANO,ESTADO,COMBUSTÍVEL,unit,created_at,month,volume
0,2000.0,RONDÔNIA,GASOLINA C (m3),m3,2023-10-10 13:07:50.805680,Jan,136073.253
1,2000.0,ACRE,GASOLINA C (m3),m3,2023-10-10 13:07:50.805680,Jan,3358.346
2,2000.0,AMAZONAS,GASOLINA C (m3),m3,2023-10-10 13:07:50.805680,Jan,20766.918
3,2000.0,RORAIMA,GASOLINA C (m3),m3,2023-10-10 13:07:50.805680,Jan,3716.032
4,2000.0,PARÁ,GASOLINA C (m3),m3,2023-10-10 13:07:50.805680,Jan,29755.907
...,...,...,...,...,...,...,...
54427,2020.0,RIO GRANDE DO SUL,GLP (m3),m3,2023-10-10 13:07:50.805680,Dez,64045.161232
54428,2020.0,MATO GROSSO DO SUL,GLP (m3),m3,2023-10-10 13:07:50.805680,Dez,16281.139493
54429,2020.0,MATO GROSSO,GLP (m3),m3,2023-10-10 13:07:50.805680,Dez,18321.987319
54430,2020.0,GOIÁS,GLP (m3),m3,2023-10-10 13:07:50.805680,Dez,46850.585145


In [66]:
df.replace({"month": MONTH_DICT})

,ANO,ESTADO,COMBUSTÍVEL,unit,created_at,month,volume
0,2000.0,RONDÔNIA,GASOLINA C (m3),m3,2023-10-10 13:07:50.805680,1,136073.253
1,2000.0,ACRE,GASOLINA C (m3),m3,2023-10-10 13:07:50.805680,1,3358.346
2,2000.0,AMAZONAS,GASOLINA C (m3),m3,2023-10-10 13:07:50.805680,1,20766.918
3,2000.0,RORAIMA,GASOLINA C (m3),m3,2023-10-10 13:07:50.805680,1,3716.032
4,2000.0,PARÁ,GASOLINA C (m3),m3,2023-10-10 13:07:50.805680,1,29755.907
...,...,...,...,...,...,...,...
54427,2020.0,RIO GRANDE DO SUL,GLP (m3),m3,2023-10-10 13:07:50.805680,12,64045.161232
54428,2020.0,MATO GROSSO DO SUL,GLP (m3),m3,2023-10-10 13:07:50.805680,12,16281.139493
54429,2020.0,MATO GROSSO,GLP (m3),m3,2023-10-10 13:07:50.805680,12,18321.987319
54430,2020.0,GOIÁS,GLP (m3),m3,2023-10-10 13:07:50.805680,12,46850.585145


In [106]:
df.dtypes

year_month    datetime64[ns]
uf                    object
product               object
unit                  object
volume                object
created_at    datetime64[ns]
dtype: object

In [109]:
df['uf'] = df['uf'].astype('string')
df['product'] = df['product'].astype('string')
df['unit'] = df['unit'].astype('string')

/tmp/ipykernel_7353/1726659777.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['uf'] = df['uf'].astype('string')
/tmp/ipykernel_7353/1726659777.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['product'] = df['product'].astype('string')
/tmp/ipykernel_7353/1726659777.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [128]:
df.dtypes

year_month    datetime64[ns]
uf                    string
product               string
unit                  string
volume               float64
created_at    datetime64[ns]
dtype: object

In [127]:
# df['volume'] = df['volume'].astype(float)
# df['volume'] = df['volume'].astype(float)
df['volume'] = pd.to_numeric(df['volume'])


/tmp/ipykernel_7353/2029536521.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['volume'] = pd.to_numeric(df['volume'])


In [116]:
import numpy as np
# df[column] = df[column].astype(np.float64)

# df['volume'] = df['volume'].astype(np.float64)

ValueError: could not convert string to float: ''

In [137]:
df

,year_month,uf,product,unit,volume,created_at
0,2000-01-01,RONDÔNIA,GASOLINA C (m3),m3,136073.253000,2023-10-10 13:32:01.584206
1,2000-01-01,ACRE,GASOLINA C (m3),m3,3358.346000,2023-10-10 13:32:01.584206
2,2000-01-01,AMAZONAS,GASOLINA C (m3),m3,20766.918000,2023-10-10 13:32:01.584206
3,2000-01-01,RORAIMA,GASOLINA C (m3),m3,3716.032000,2023-10-10 13:32:01.584206
4,2000-01-01,PARÁ,GASOLINA C (m3),m3,29755.907000,2023-10-10 13:32:01.584206
...,...,...,...,...,...,...
54427,2020-12-01,RIO GRANDE DO SUL,GLP (m3),m3,64045.161232,2023-10-10 13:32:01.584206
54428,2020-12-01,MATO GROSSO DO SUL,GLP (m3),m3,16281.139493,2023-10-10 13:32:01.584206
54429,2020-12-01,MATO GROSSO,GLP (m3),m3,18321.987319,2023-10-10 13:32:01.584206
54430,2020-12-01,GOIÁS,GLP (m3),m3,46850.585145,2023-10-10 13:32:01.584206


In [139]:
output_dir = '../data/partitioned_parquet_data'

# Specify the columns to use for partitioning
partition_cols = ['year_month']

# Save the DataFrame to Parquet with partitioning
df.to_parquet(output_dir, partition_cols=partition_cols, engine='pyarrow',compression='snappy')


In [130]:
import pyarrow as pa
import pyarrow.parquet as pq

In [131]:
tab = pa.Table.from_pandas(df)


In [140]:
pq.write_table(tab, file_path, compression='SNAPPY')


In [141]:
pq.write_table(tab, './df_pq_snappy', compression='SNAPPY')

In [143]:
df.to_parquet('./df_to_parquet_p_uf',partition_cols=['uf'],engine='pyarrow')
# df.to_parquet(output_dir, partition_cols=partition_cols, engine='pyarrow')


In [145]:
import pandas as pd

In [146]:
df_oil = pd.read_parquet('../data/oil_parquet')

In [147]:
df_oil

,year_month,uf,product,unit,volume,created_at
0,2000-01-01,RONDÔNIA,GASOLINA C (m3),m3,136073.253000,2023-10-10 21:15:12.988322
1,2000-01-01,ACRE,GASOLINA C (m3),m3,3358.346000,2023-10-10 21:15:12.988322
2,2000-01-01,AMAZONAS,GASOLINA C (m3),m3,20766.918000,2023-10-10 21:15:12.988322
3,2000-01-01,RORAIMA,GASOLINA C (m3),m3,3716.032000,2023-10-10 21:15:12.988322
4,2000-01-01,PARÁ,GASOLINA C (m3),m3,29755.907000,2023-10-10 21:15:12.988322
...,...,...,...,...,...,...
54427,2020-12-01,RIO GRANDE DO SUL,GLP (m3),m3,64045.161232,2023-10-10 21:15:12.988322
54428,2020-12-01,MATO GROSSO DO SUL,GLP (m3),m3,16281.139493,2023-10-10 21:15:12.988322
54429,2020-12-01,MATO GROSSO,GLP (m3),m3,18321.987319,2023-10-10 21:15:12.988322
54430,2020-12-01,GOIÁS,GLP (m3),m3,46850.585145,2023-10-10 21:15:12.988322


In [148]:
df_oil = df_oil[['year_month','volume']]

In [149]:
# df_oil = df_oil.groupby('year_month').sum()

In [150]:
df_oil['ano'] = df_oil['year_month'].dt.year

/tmp/ipykernel_57992/2048226672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_oil['ano'] = df_oil['year_month'].dt.year


In [136]:
df_oil.drop(columns=['year_month'],inplace=True)
# df_oil_acre.drop(columns=['mês'],inplace=True)

/tmp/ipykernel_57992/3490461264.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_oil.drop(columns=['year_month'],inplace=True)


In [137]:
# df_oil.drop
df_oil = df_oil.pivot_table(columns='ano',aggfunc='sum')

In [138]:
s_oil_parquet = df_oil.iloc[:].squeeze()


In [139]:
s_oil_parquet

ano
2000    1.615858e+08
2001    1.679425e+08
2002    1.635339e+08
2003    1.529466e+08
2004    1.644017e+08
2005    1.651756e+08
2006    1.649343e+08
2007    1.814908e+08
2008    1.966418e+08
2009    2.002764e+08
2010    2.126558e+08
2011    2.305579e+08
2012    2.395704e+08
2013    2.462708e+08
2014    2.707333e+08
2015    2.638810e+08
2016    2.476081e+08
2017    2.513247e+08
2018    2.543917e+08
2019    2.579250e+08
2020    1.777013e+08
Name: volume, dtype: float64

In [104]:
new_index = [i for i in range(0,21)]

In [107]:
new_index

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [110]:
# s_oil_parquet = s_oil_parquet.set_index(pd.Index(new_index))
new_s = pd.Series(s_oil_parquet.values, index=new_index)
new_s


0     1.615858e+08
1     1.679425e+08
2     1.635339e+08
3     1.529466e+08
4     1.644017e+08
5     1.651756e+08
6     1.649343e+08
7     1.814908e+08
8     1.966418e+08
9     2.002764e+08
10    2.126558e+08
11    2.305579e+08
12    2.395704e+08
13    2.462708e+08
14    2.707333e+08
15    2.638810e+08
16    2.476081e+08
17    2.513247e+08
18    2.543917e+08
19    2.579250e+08
20    1.777013e+08
dtype: float64

In [158]:
df_total = pd.read_excel('/home/felipe/Desktop/vendas-combustiveis-m3.xlsx',sheet_name='Plan1',usecols='C:W', nrows=12, skiprows=52)

In [159]:
l = [2000+i for i in range(0,21)]
string_list = []
for year in l:
    string_list.append(str(year))

In [160]:
string_list

['2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020']

In [161]:
df_total.columns = string_list

In [162]:
df_total

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,1.364637e+07,1.853120e+07,1.121790e+07,1.265756e+07,1.560447e+07,1.261595e+07,1.211656e+07,1.721390e+07,1.503118e+07,1.547539e+07,...,2.248175e+07,1.487028e+07,2.118626e+07,2.943194e+07,1.811200e+07,1.884760e+07,2.357775e+07,2.134261e+07,1.834297e+07,1.513507e+07
1,9.841822e+06,1.376793e+07,1.852239e+07,1.073606e+07,1.324108e+07,1.565201e+07,1.283656e+07,1.277396e+07,1.819305e+07,1.542920e+07,...,1.912445e+07,2.306004e+07,1.577436e+07,2.240024e+07,2.936584e+07,1.695271e+07,1.900380e+07,2.365585e+07,2.159468e+07,1.230695e+07
2,1.370608e+07,9.756567e+06,1.375953e+07,1.763635e+07,1.115225e+07,1.324192e+07,1.595795e+07,1.366311e+07,1.361676e+07,1.816018e+07,...,1.764299e+07,2.035340e+07,2.462975e+07,1.669280e+07,2.167740e+07,2.784652e+07,1.668818e+07,1.926030e+07,2.427136e+07,1.485362e+07
3,1.346811e+07,1.322115e+07,9.766751e+06,1.315239e+07,1.854358e+07,1.139077e+07,1.363360e+07,1.711134e+07,1.519061e+07,1.404094e+07,...,1.734328e+07,1.855040e+07,2.172204e+07,2.649857e+07,1.590346e+07,2.089710e+07,2.840659e+07,1.672655e+07,1.965157e+07,1.639673e+07
4,1.370660e+07,1.341948e+07,1.336732e+07,9.368968e+06,1.384793e+07,1.858985e+07,1.154706e+07,1.429104e+07,1.826586e+07,1.568725e+07,...,2.169014e+07,1.855305e+07,1.966934e+07,2.311142e+07,2.763880e+07,1.528184e+07,2.133298e+07,2.897393e+07,1.703417e+07,1.328373e+07
5,1.290753e+07,1.381585e+07,1.319785e+07,1.261206e+07,9.785203e+06,1.385368e+07,1.864501e+07,1.241731e+07,1.529488e+07,1.860289e+07,...,1.589281e+07,2.336033e+07,1.982588e+07,2.066828e+07,2.202296e+07,2.626806e+07,1.543625e+07,2.114166e+07,3.001357e+07,1.121496e+07
6,1.749255e+07,1.260680e+07,1.333381e+07,1.272419e+07,1.413694e+07,1.001364e+07,1.410380e+07,2.019889e+07,1.358489e+07,1.531583e+07,...,1.824462e+07,1.713523e+07,2.428374e+07,2.115485e+07,2.009835e+07,2.075940e+07,2.576730e+07,1.561067e+07,2.156370e+07,2.032894e+07
7,1.451603e+07,1.746543e+07,1.290347e+07,1.268089e+07,1.362230e+07,1.399121e+07,1.022714e+07,1.514754e+07,2.189679e+07,1.410064e+07,...,2.146219e+07,1.929859e+07,1.779962e+07,2.528347e+07,2.082753e+07,1.942741e+07,2.039485e+07,2.760064e+07,1.628633e+07,1.507544e+07
8,1.223170e+07,1.406884e+07,1.673292e+07,1.209293e+07,1.329261e+07,1.356369e+07,1.555821e+07,1.115884e+07,1.623126e+07,2.231086e+07,...,1.763436e+07,2.247678e+07,2.053284e+07,1.863108e+07,2.408841e+07,1.924144e+07,1.919335e+07,1.994325e+07,2.961373e+07,1.070232e+07
9,1.521019e+07,1.217792e+07,1.335596e+07,1.554312e+07,1.260281e+07,1.322867e+07,1.402779e+07,1.799727e+07,1.224286e+07,1.619390e+07,...,1.572959e+07,1.840766e+07,2.377483e+07,2.167272e+07,1.844165e+07,2.365486e+07,1.915758e+07,1.946673e+07,2.032010e+07,2.017455e+07


In [163]:
df_s =[]

In [164]:
for col in df_total.columns:
    # print(col)
    # print(type(col))
    df_s.append(df_total[col].sum())


In [165]:
df_ss = pd.Series(df_s)
df_ss = df_ss.rename_axis('Year')
df_ss.rename('Volume Total')
df_ss

Year
0     1.615858e+08
1     1.679425e+08
2     1.635339e+08
3     1.529466e+08
4     1.644017e+08
5     1.651756e+08
6     1.649343e+08
7     1.814908e+08
8     1.966418e+08
9     2.002764e+08
10    2.126558e+08
11    2.305579e+08
12    2.395704e+08
13    2.462708e+08
14    2.707333e+08
15    2.638810e+08
16    2.476081e+08
17    2.513247e+08
18    2.543917e+08
19    2.579250e+08
20    1.777013e+08
dtype: float64

In [166]:
relative_dif = (new_s - df_ss)/df_ss

In [167]:
relative_dif

0    -9.221824e-16
1    -7.098220e-16
2     7.289579e-16
3    -3.897090e-16
4    -9.063873e-16
5    -5.412842e-16
6    -1.264845e-15
7     6.568339e-16
8    -1.515564e-16
9     5.952240e-16
10    9.810041e-16
11   -3.877853e-16
12   -4.975960e-16
13   -3.630433e-16
14    0.000000e+00
15   -3.388155e-16
16   -1.203608e-16
17   -2.371619e-16
18    0.000000e+00
19   -6.932789e-16
20   -1.677102e-16
dtype: float64

In [168]:
greater_than_minimum = dif < 0.0000001
greater_than_minimum.sum() == 21


True

In [26]:
df_total = df_total.rename_axis('volume')

In [37]:
13646370.0414861+9841822.23001855+13706076.1982171+13468112.6608275+13706596.1938163+12907529.0797421+17492550.3301299+14516026.2260056+12231702.2010891+15210190.1900501+13756003.61218+11102857.3312356


161585836.29479793

In [38]:
161585836.29479793 == 

False

In [39]:
1.615858e+08	

161585800.0